In [5]:
import os
from pathlib import Path

from jmetal.core.algorithm import Algorithm
from jmetal.core.quality_indicator import *
from jmetal.util.solution import print_function_values_to_file, print_variables_to_file, read_solutions

In [19]:
def generate_summary_from_experiment(input_dir, quality_indicators,
                                     reference_fronts = ''):
    reference_change = 2500
    ref_time = 1
    if not quality_indicators:
        quality_indicators = []
        
    with open('QualityIndicatorSummary.csv', 'w+') as of:
        of.write('Algorithm,Problem,ExecutionId,IndicatorName,IndicatorValue\n')

    for dirname, _, filenames in os.walk(input_dir):
        print(dirname)
        for filename in sorted(filenames):
            try:
                # Linux filesystem
                algorithm, problem = dirname.split('/')[-2:]
            except ValueError:
                # Windows filesystem
                algorithm, problem = dirname.split('\\')[-2:]

            if 'FUN' in filename and problem == "FDA3":
                solutions = read_solutions(os.path.join(dirname, filename))
                run_tag = [s for s in filename.split('.') if s.isdigit()].pop()
                print(filename)
                for indicator in quality_indicators:
                    ref_time = min(int(int(run_tag)/reference_change) + 1, 10)

                    reference_front_file = "FDA3_time{}.pf".format(ref_time) 
                    print("file{} - {}".format(filename, ref_time))    
                    # Add reference front if any
                    if hasattr(indicator, 'reference_front'):
                        if Path(reference_front_file).is_file():
                            reference_front = []
                            with open(reference_front_file) as file:
                                for line in file:
                                    reference_front.append([float(x) for x in line.split()])

                            indicator.reference_front = reference_front
                        else:
                            print("no reference front for {}".format(problem))

                    result = indicator.compute([solutions[i].objectives for i in range(len(solutions))])

                    # Save quality indicator value to file
                    with open('QualityIndicatorSummary.csv', 'a+') as of:
                        of.write(','.join([algorithm, problem, run_tag, indicator.get_short_name(), str(result)]))
                        of.write('\n')

In [20]:
output_directory = "data-500pop-100000evals-1runs-fda-tau5"
reference_front = "FDA3_time0.pf"

generate_summary_from_experiment(output_directory, [InvertedGenerationalDistance(None)], reference_front)

data-500pop-100000evals-1runs-fda-tau5
data-500pop-100000evals-1runs-fda-tau5/MOEAD
data-500pop-100000evals-1runs-fda-tau5/MOEAD/FDA2
data-500pop-100000evals-1runs-fda-tau5/MOEAD/FDA3
FUN.0.tsv
fileFUN.0.tsv - 1
FUN.0.tsv.1000
fileFUN.0.tsv.1000 - 1
FUN.0.tsv.10000
fileFUN.0.tsv.10000 - 5
FUN.0.tsv.11000
fileFUN.0.tsv.11000 - 5
FUN.0.tsv.12000
fileFUN.0.tsv.12000 - 5
FUN.0.tsv.13000
fileFUN.0.tsv.13000 - 6
FUN.0.tsv.14000
fileFUN.0.tsv.14000 - 6
FUN.0.tsv.15000
fileFUN.0.tsv.15000 - 7
FUN.0.tsv.16000
fileFUN.0.tsv.16000 - 7
FUN.0.tsv.17000
fileFUN.0.tsv.17000 - 7
FUN.0.tsv.18000
fileFUN.0.tsv.18000 - 8
FUN.0.tsv.19000
fileFUN.0.tsv.19000 - 8
FUN.0.tsv.2000
fileFUN.0.tsv.2000 - 1
FUN.0.tsv.20000
fileFUN.0.tsv.20000 - 9
FUN.0.tsv.21000
fileFUN.0.tsv.21000 - 9
FUN.0.tsv.22000
fileFUN.0.tsv.22000 - 9
FUN.0.tsv.23000
fileFUN.0.tsv.23000 - 10
FUN.0.tsv.24000
fileFUN.0.tsv.24000 - 10
FUN.0.tsv.25000
fileFUN.0.tsv.25000 - 10
FUN.0.tsv.26000
fileFUN.0.tsv.26000 - 10
FUN.0.tsv.27000
fileFUN.0.ts